In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from layers import *

In [3]:
X = np.random.normal(size=[128, 32, 32, 16])

In [4]:
inputs = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 16])

In [5]:
# def architecture(inputs, training):
#     P = 0.5
#     L = 54
#     l = 1
    
#     out = first_layer(inputs, training, 'input')

#     with tf.variable_scope('stack1', reuse=tf.AUTO_REUSE, initializer=tf.global_variables_initializer()):
#         for i in range(1, 19):
#             p = 1 - l/L * (1 - P)
#             out = residual_block(out, 16, p, training, 'res'+str(i))
#             l += 1

#     with tf.variable_scope('stack2', reuse=tf.AUTO_REUSE, initializer=tf.global_variables_initializer()):
#         p = 1 - l/L * (1 - P)
#         out = transition_block(out, 32, p, training, 'res'+str(1))
#         l += 1
#         for i in range(2, 19):
#             p = 1 - l/L * (1 - P)
#             out = residual_block(out, 32, p, training, 'res'+str(i))
#             l += 1

#     with tf.variable_scope('stack3', reuse=tf.AUTO_REUSE, initializer=tf.global_variables_initializer()):
#         p = 1 - l/L * (1 - P)
#         out = transition_block(out, 64, p, training, 'res'+str(1))
#         l += 1
#         for i in range(2, 19):
#             p = 1 - l/L * (1 - P)
#             out = residual_block(out, 64, p, training, 'res'+str(i)) 

#     return output_layer(out, 'out', 10)

1. dynamic computation graph library
2. change code such that if else logic is within Session

In [6]:
def architecture(inputs, training):
    P = 0.5
    L = 54
    l = 1

    out = first_layer(inputs, training, 'input')

    with tf.variable_scope('stack1', reuse=tf.AUTO_REUSE):
        for i in range(1, 19):
            p = 1 - l/L * (1 - P)
            bernoulli = np.random.uniform()
            survives = tf.less(bernoulli, p)
            out = tf.cond(survives,
                          lambda: residual_block(out, 16, 1, training, 'res_surv'+str(i)),
                          lambda: residual_block(out, 16, 0, training, 'res_not_surv'+str(i)))
            l += 1

    with tf.variable_scope('stack2', reuse=tf.AUTO_REUSE):
        p = 1 - l/L * (1 - P)
        bernoulli = np.random.uniform()
        survives = tf.less(bernoulli, p)
        out = tf.cond(survives,
                      lambda: transition_block(out, 32, 1, training, 'res_surv'+str(1)),
                      lambda: transition_block(out, 32, 0, training, 'res_not_surv'+str(1)))
        l += 1

        for i in range(2, 19):
            p = 1 - l/L * (1 - P)
            bernoulli = np.random.uniform()
            survives = tf.less(bernoulli, p)
            out = tf.cond(survives,
                          lambda: residual_block(out, 32, 1, training, 'res_surv'+str(i)),
                          lambda: residual_block(out, 32, 0, training, 'res_not_surv'+str(i)))
            l += 1

    with tf.variable_scope('stack3', reuse=tf.AUTO_REUSE):
        p = 1 - l/L * (1 - P)
        bernoulli = np.random.uniform()
        survives = tf.less(bernoulli, p)
        out = tf.cond(survives,
                      lambda: transition_block(out, 64, 1, training, 'res_surv'+str(1)),
                      lambda: transition_block(out, 64, 0, training, 'res_not_surv'+str(1)))
        l += 1

        for i in range(2, 19):
            p = 1 - l/L * (1 - P)
            bernoulli = np.random.uniform()
            survives = tf.less(bernoulli, p)
            out = tf.cond(survives,
                          lambda: residual_block(out, 64, 1, training, 'res_surv'+str(i)),
                          lambda: residual_block(out, 64, 0, training, 'res_not_surv'+str(i)))

    return output_layer(out, 'out', 10)

In [7]:
with tf.Session() as sess:
#     writer = tf.summary.FileWriter('logs', sess.graph)
    ls = []
    for i in range(2):
        with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
            out = architecture(inputs, True)
            if i == 0:
                sess.run(tf.global_variables_initializer())
            print(i)
            ls.append(sess.run(out, feed_dict={inputs: X}))
#     writer.close()

0
1
